# Indexed Expressions and Tensor Utilities

## Author: Zach Etienne

## This notebook explores the main features of `indexedexp.py`, NRPy's toolkit for working with indexed expressions such as vectors, matrices, tensors, and Levi-Civita objects.
We will:
- Build symbolic tensors of various ranks and dimensions.
- Declare indexed expressions with built in symmetry.
- Create zero tensors and detect tensor rank from list nesting.
- Enforce symmetry axes to zero out derivative components.
- Invert symmetric and generic matrices.
- Construct Levi-Civita symbols and tensors in 3 dimensions.
- Put the pieces together in a small worked example.

The focus is on short, concrete examples that you can tweak and extend.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, use for reference; you should be able to pick up the syntax as you follow the tutorial.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**

### NRPy Source Code for this module:  
* [indexedexp.py](../edit/indexedexp.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python/NRPy-modules): Initialize core Python/NRPy modules
1. [Step 2](#Step-2:-Creating-symbolic-tensors-with-create_tensor_symbolic()): Creating symbolic tensors with `create_tensor_symbolic()`
1. [Step 3](#Step-3:-Declaring-indexed-expressions-with-declare_indexedexp()-and-declarerank*()): Declaring indexed expressions with `declare_indexedexp()` and `declarerank*()`
1. [Step 4](#Step-4:-Working-with-tensor-symmetries-in-ranks-2,-3,-and-4): Working with tensor symmetries in ranks 2, 3, and 4
1. [Step 5](#Step-5:-Zero-tensors-and-rank-detection-with-zerorank*()-and-get_rank()): Zero tensors and rank detection with `zerorank*()` and `get_rank()`
1. [Step 6](#Step-6:-Symmetry-axes-and-zeroing-derivatives-across-symmetry-surfaces): Symmetry axes and zeroing derivatives across symmetry surfaces
1. [Step 7](#Step-7:-Inverting-matrices-with-symmetric-and-generic-matrix-inverters): Inverting matrices with symmetric and generic matrix inverters
1. [Step 8](#Step-8:-Levi-Civita-symbols-and-tensors-in-3-dimensions): Levi-Civita symbols and tensors in 3 dimensions
1. [Step 9](#Step-9:-Worked-example:-3-metric,-its-inverse,-and-Levi-Civita-tensor): Worked example: 3-metric, its inverse, and Levi-Civita tensor

# Step 1: Initialize core Python/NRPy modules
### \[Back to [top](#Table-of-Contents)\]

We start by importing the modules that will be used in every step:

- `nrpy.indexedexp` as `ixp`: the main tensor utility module.
- `nrpy.params` as `par`: NRPy's parameter interface, used here for symmetry axes.
- `nrpy.helpers.functional` as `func`: small helpers such as cartesian products and flattening.
- `sympy` as `sp`: the symbolic math engine.

In [1]:
# Step 1: Initialize core Python/NRPy modules
import sympy as sp

import nrpy.indexedexp as ixp          # NRPy: indexed expressions and tensor utilities
import nrpy.params as par              # NRPy: parameter interface
import nrpy.helpers.functional as func # NRPy: functional utilities (e.g., product, flatten)

# Quick SymPy sanity check
x = sp.Symbol("x")
print("SymPy sanity check: d/dx of x**3 =", sp.diff(x**3, x))

SymPy sanity check: d/dx of x**3 = 3*x**2


# Step 2: Creating symbolic tensors with create_tensor_symbolic()
### \[Back to [top](#Table-of-Contents)\]

The low level building block in `indexedexp.py` is `create_tensor_symbolic()`.
It constructs nested Python lists of SymPy symbols with a given shape and naming convention:

```python
tensor = ixp.create_tensor_symbolic(shape, symbol,
                                    preindex=None,
                                    character_zero_index="")
```

- `shape`:
  - A single integer gives a rank 1 object (a vector).
  - A list such as `[3, 3]` gives a matrix.
  - `[2, 2, 2]` would give a rank 3 tensor with eight entries.
- `symbol`:
  - Base name for all components, for example `"a"` gives symbols like `a0`, `a01`, and so on.
- `preindex`:
  - Optional list of integers that gets prefixed into the index string.
- `character_zero_index`:
  - When nonempty, indices are letters instead of digits, starting from that character.

In [2]:
# Step 2: Creating symbolic tensors with create_tensor_symbolic()

# Example 2a: rank 1 tensor (vector) with length 3
v = ixp.create_tensor_symbolic(3, "v")
print("Rank 1 tensor v:")
print("  v =", v)
print()

# Example 2b: rank 2 tensor (2 by 2 matrix)
A = ixp.create_tensor_symbolic([2, 2], "A")
print("Rank 2 tensor A (2 by 2):")
for row in A:
    print(" ", row)
print("Single entry A[1][0] =", A[1][0])
print()

# Example 2c: rank 3 tensor (2 by 2 by 2)
T = ixp.create_tensor_symbolic([2, 2, 2], "T")
print("Rank 3 tensor T, showing a few components:")
print("  T[0][0][0] =", T[0][0][0])
print("  T[1][0][1] =", T[1][0][1])
print("  T[1][1][1] =", T[1][1][1])
print()

# Example 2d: using preindex to prepend digits to indices
B = ixp.create_tensor_symbolic([2, 2], "B", preindex=[3, 1])
print("Tensor B with preindex [3, 1]:")
for row in B:
    print(" ", row)
print()

# Example 2e: using character_zero_index for letter based indexing
C = ixp.create_tensor_symbolic([2, 2], "C", character_zero_index="A")
print("Tensor C with character_zero_index='A':")
for row in C:
    print(" ", row)
print()

# Example 2f: combining preindex and character_zero_index
D = ixp.create_tensor_symbolic([2, 2], "D", preindex=[1, 2], character_zero_index="x")
print("Tensor D with preindex [1, 2] and character_zero_index='x':")
for row in D:
    print(" ", row)

Rank 1 tensor v:
  v = [v0, v1, v2]

Rank 2 tensor A (2 by 2):
  [A00, A01]
  [A10, A11]
Single entry A[1][0] = A10

Rank 3 tensor T, showing a few components:
  T[0][0][0] = T000
  T[1][0][1] = T101
  T[1][1][1] = T111

Tensor B with preindex [3, 1]:
  [B3100, B3101]
  [B3110, B3111]

Tensor C with character_zero_index='A':
  [CAA, CAB]
  [CBA, CBB]

Tensor D with preindex [1, 2] and character_zero_index='x':
  [Dyzxx, Dyzxy]
  [Dyzyx, Dyzyy]


# Step 3: Declaring indexed expressions with declare_indexedexp() and declarerank*()
### \[Back to [top](#Table-of-Contents)\]

While `create_tensor_symbolic()` is very general, most of the time you will want
a higher level interface that:

- Chooses a rank and dimension.
- Applies symmetries.
- Is easy to read in physics notation.

For this, `indexedexp.py` provides:

- `ixp.declarerank1(basename, dimension=...)`
- `ixp.declarerank2(basename, dimension=..., symmetry=...)`
- `ixp.declarerank3(...)`
- `ixp.declarerank4(...)`

All of these call the core routine:

```python
ixp.declare_indexedexp(basename, rank=..., dimension=..., symmetry=...)
```

We now declare a few simple objects and inspect their structure.

In [3]:
# Step 3: Declaring indexed expressions with declarerank*()

# Example 3a: rank 1 vector of dimension 3
vU = ixp.declarerank1("vU", dimension=3)
print("Rank 1 vector vU (dimension 3):")
print("  vU =", vU)
print("  Types:", [type(entry) for entry in vU])
print()

# Example 3b: rank 2 matrix with no symmetry
MDD = ixp.declarerank2("MDD", dimension=3)
print("Rank 2 tensor MDD (no symmetry, dimension 3):")
for row in MDD:
    print(" ", row)
print()

# Example 3c: symmetric rank 2 tensor with symmetry='sym01'
gDD = ixp.declarerank2("gDD", dimension=3, symmetry="sym01")
print("Symmetric rank 2 tensor gDD (symmetry='sym01'):")
for row in gDD:
    print(" ", row)
print("Check symmetry: gDD[0][1] and gDD[1][0]:")
print(" ", gDD[0][1], "and", gDD[1][0])
print()

# Example 3d: rank 3 tensor with symmetry between indices 0 and 1
TDDD = ixp.declarerank3("TDDD", dimension=3, symmetry="sym01")
print("Rank 3 tensor TDDD, symmetry='sym01': TDDD[0][1][2] and TDDD[1][0][2]:")
print(" ", TDDD[0][1][2], "and", TDDD[1][0][2])
print()

# Example 3e: rank 4 tensor with two symmetry blocks (0-1) and (2-3)
R4DDDD = ixp.declarerank4("R4DDDD", dimension=3, symmetry="sym01_sym23")
print("Rank 4 tensor R4DDDD, symmetry='sym01_sym23':")
print("  R4DDDD[0][1][0][1] =", R4DDDD[0][1][0][1])
print("  R4DDDD[1][0][0][1] =", R4DDDD[1][0][0][1], "(symmetry in indices 0 and 1)")
print("  R4DDDD[0][1][1][2] =", R4DDDD[0][1][1][2])
print("  R4DDDD[0][1][2][1] =", R4DDDD[0][1][2][1], "(symmetry in indices 2 and 3)")
print()

# Example 3f: equivalent call using declare_indexedexp() directly
TDD_direct = ixp.declare_indexedexp("TDD", rank=2, dimension=2, symmetry="sym01")
print("TDD from declare_indexedexp directly (rank=2, dimension=2, symmetry='sym01'):")
for row in TDD_direct:
    print(" ", row)

Rank 1 vector vU (dimension 3):
  vU = [vU0, vU1, vU2]
  Types: [<class 'sympy.core.symbol.Symbol'>, <class 'sympy.core.symbol.Symbol'>, <class 'sympy.core.symbol.Symbol'>]

Rank 2 tensor MDD (no symmetry, dimension 3):
  [MDD00, MDD01, MDD02]
  [MDD10, MDD11, MDD12]
  [MDD20, MDD21, MDD22]

Symmetric rank 2 tensor gDD (symmetry='sym01'):
  [gDD00, gDD01, gDD02]
  [gDD01, gDD11, gDD12]
  [gDD02, gDD12, gDD22]
Check symmetry: gDD[0][1] and gDD[1][0]:
  gDD01 and gDD01

Rank 3 tensor TDDD, symmetry='sym01': TDDD[0][1][2] and TDDD[1][0][2]:
  TDDD012 and TDDD012

Rank 4 tensor R4DDDD, symmetry='sym01_sym23':
  R4DDDD[0][1][0][1] = R4DDDD0101
  R4DDDD[1][0][0][1] = R4DDDD0101 (symmetry in indices 0 and 1)
  R4DDDD[0][1][1][2] = R4DDDD0112
  R4DDDD[0][1][2][1] = R4DDDD0112 (symmetry in indices 2 and 3)

TDD from declare_indexedexp directly (rank=2, dimension=2, symmetry='sym01'):
  [TDD00, TDD01]
  [TDD01, TDD11]


# Step 4: Working with tensor symmetries in ranks 2, 3, and 4
### \[Back to [top](#Table-of-Contents)\]

The `symmetry` keyword controls how indices are related.  
Supported patterns are based on strings like:

- `"sym01"`: symmetric in indices 0 and 1.
- `"sym12"`: symmetric in indices 1 and 2.
- `"sym01_sym23"`: symmetric in 0-1 and separately in 2-3 (for rank 4).
- `"anti01"`: antisymmetric in indices 0 and 1.
- `"nosym"`: no symmetry.

Internally, `indexedexp.py` expands more complex combinations such as `"sym012"`
into several pairwise symmetries. The end result is that many components are
automatically identified with each other, and some are set to zero in the
antisymmetric case.

In this step we:

- Count how many independent components remain after symmetry is applied.
- Check a couple of symmetry relations explicitly.

In [4]:
# Step 4: Working with tensor symmetries

def flatten_rankN(tensor, times):
    """
    Repeatedly flatten a nested list 'times' times using func.flatten.
    Ensure the result is a concrete list so len(), set(), etc. work.
    """
    out = tensor
    for _ in range(times):
        out = list(func.flatten(out))
    return out

# Example 4a: rank 2 symmetric tensor in 3 dimensions
M_sym = ixp.declarerank2("M", dimension=3, symmetry="sym01")
flat = flatten_rankN(M_sym, 1)
unique_symbols = set(flat)
print("Rank 2 symmetric (sym01), dimension 3:")
print("  Total components:", len(flat))
print("  Unique symbols  :", len(unique_symbols))
print()

# Example 4b: rank 3 symmetry patterns in 3 dimensions
for sym in ["sym01", "sym02", "sym12", "sym012"]:
    T_sym = ixp.declarerank3("T", dimension=3, symmetry=sym)
    flat = flatten_rankN(T_sym, 2)
    unique_count = len(set(flat))
    print(f"Rank 3 tensor, symmetry='{sym}': unique components = {unique_count}")
print()

# Example 4c: rank 4 symmetry patterns in 3 dimensions
sym_list_rank4 = [
    "sym01", "sym02", "sym03", "sym12", "sym13", "sym23",
    "sym012", "sym013", "sym01_sym23", "sym02_sym13",
    "sym023", "sym03_sym12", "sym123", "sym0123",
]
for sym in sym_list_rank4:
    R_sym = ixp.declarerank4("R", dimension=3, symmetry=sym)
    flat = flatten_rankN(R_sym, 3)
    unique_count = len(set(flat))
    print(f"Rank 4 tensor, symmetry='{sym:10s}': unique components = {unique_count}")
print()

# Example 4d: antisymmetric rank 2 tensor in 3 dimensions
A_anti = ixp.declarerank2("A", dimension=3, symmetry="anti01")
flat_anti = flatten_rankN(A_anti, 1)

# Ignore signs when counting; antisymmetry shows up as plus/minus.
abs_nonzero = set(map(abs, flat_anti)) - {0}
print("Rank 2 antisymmetric (anti01):")
print("  Unique nonzero symbols up to sign:", abs_nonzero)
print("  Check antisymmetry: A[1][2] and -A[2][1]:")
print("   ", A_anti[1][2], "and", -A_anti[2][1])

Rank 2 symmetric (sym01), dimension 3:
  Total components: 9
  Unique symbols  : 6

Rank 3 tensor, symmetry='sym01': unique components = 18
Rank 3 tensor, symmetry='sym02': unique components = 18
Rank 3 tensor, symmetry='sym12': unique components = 18
Rank 3 tensor, symmetry='sym012': unique components = 10

Rank 4 tensor, symmetry='sym01     ': unique components = 54
Rank 4 tensor, symmetry='sym02     ': unique components = 54
Rank 4 tensor, symmetry='sym03     ': unique components = 54
Rank 4 tensor, symmetry='sym12     ': unique components = 54
Rank 4 tensor, symmetry='sym13     ': unique components = 54
Rank 4 tensor, symmetry='sym23     ': unique components = 54
Rank 4 tensor, symmetry='sym012    ': unique components = 30
Rank 4 tensor, symmetry='sym013    ': unique components = 30
Rank 4 tensor, symmetry='sym01_sym23': unique components = 36
Rank 4 tensor, symmetry='sym02_sym13': unique components = 36
Rank 4 tensor, symmetry='sym023    ': unique components = 30
Rank 4 tensor, sy

# Step 5: Zero tensors and rank detection with zerorank*() and get_rank()
### \[Back to [top](#Table-of-Contents)\]

Often it is convenient to start from a tensor filled with zeros and then assign
only the components you care about. For that, `indexedexp.py` provides:

- `ixp.zerorank1(dimension)`
- `ixp.zerorank2(dimension)`
- `ixp.zerorank3(dimension)`
- `ixp.zerorank4(dimension)`

Each function returns the right nested list shape filled with SymPy zeros.

When working with generic nested lists, it is also helpful to detect the rank.
The helper `ixp.get_rank(IDX_EXPR)` does this by counting list nesting:

- A list of scalars has rank 1.
- A list of lists of scalars has rank 2.
- And so on.

In [5]:
# Step 5: Zero tensors and rank detection

# Example 5a: Create zero tensors of several ranks and inspect a few entries
zero_vec   = ixp.zerorank1(dimension=3)
zero_mat   = ixp.zerorank2(dimension=3)
zero_rank3 = ixp.zerorank3(dimension=2)

print("zero_vec =", zero_vec)
print("zero_mat first row =", zero_mat[0])
print("zero_rank3[0][0] =", zero_rank3[0][0])

print("\nRank detection:")
print("rank(zero_vec)   =", ixp.get_rank(zero_vec))
print("rank(zero_mat)   =", ixp.get_rank(zero_mat))
print("rank(zero_rank3) =", ixp.get_rank(zero_rank3))

# Example 5b: Flatten a rank 2 object into a simple list
# func.repeat returns a generator, so we wrap it in list(...) before calling len().
flat_zero_mat = list(func.repeat(func.flatten, zero_mat, 1))

print("\nFlattened zero_mat has length", len(flat_zero_mat))
print("All entries are SymPy zeros:", all(entry == 0 for entry in flat_zero_mat))

zero_vec = [0, 0, 0]
zero_mat first row = [0, 0, 0]
zero_rank3[0][0] = [0, 0]

Rank detection:
rank(zero_vec)   = 1
rank(zero_mat)   = 2
rank(zero_rank3) = 3

Flattened zero_mat has length 9
All entries are SymPy zeros: True


# Step 6: Symmetry axes and zeroing derivatives across symmetry surfaces
### \[Back to [top](#Table-of-Contents)\]

In many simulations, symmetry reduces the physical domain.
For example, if the solution is symmetric across the plane $z = 0$, then derivatives
with respect to the $z$ coordinate vanish on that plane.

`indexedexp.py` encodes this idea using:

1. A parameter named `indexedexp::symmetry_axes`, which lists coordinate indices that
   correspond to symmetry axes (for example `"2"` or `"01"`).
2. The helper `ixp.zero_out_derivatives_across_symmetry_axes(IDX_EXPR)`, which looks
   at the names of tensor components and sets to zero those that correspond to
   derivatives across symmetry axes.

Derivative orders are encoded in the symbol names:

- `"..._dD"` for first derivatives.
- `"..._dDD"` for second derivatives.

Higher derivative orders are not supported and will raise a `ValueError`.

In [6]:
# Step 6: Symmetry axes and derivative zeroing

# Reset symmetry axes: empty string means no symmetry constraints.
par.set_parval_from_str("indexedexp::symmetry_axes", "")

# Example 6a: No symmetry axes set
trK_dD = ixp.declarerank1("trK_dD", dimension=3)
print("Rank 1 derivative object trK_dD with no symmetry axes:")
print(" ", ixp.zero_out_derivatives_across_symmetry_axes(trK_dD))
print()

# Example 6b: Set the z axis (index 2) as a symmetry axis
par.set_parval_from_str("indexedexp::symmetry_axes", "2")

trK_dD = ixp.declarerank1("trK_dD", dimension=3)
print("Symmetry axis = 2, applied to trK_dD:")
print(" ", ixp.zero_out_derivatives_across_symmetry_axes(trK_dD))
print()

trK_dDD = ixp.declarerank2("trK_dDD", dimension=3)
print("Symmetry axis = 2, applied to trK_dDD:")
for row in ixp.zero_out_derivatives_across_symmetry_axes(trK_dDD):
    print(" ", row)
print()

# Example 6c: Multiple symmetry axes (x and y, indices 0 and 1)
par.set_parval_from_str("indexedexp::symmetry_axes", "01")

trK_dDD = ixp.declarerank2("trK_dDD", dimension=3)
print("Symmetry axes = 0,1, applied to trK_dDD:")
for row in ixp.zero_out_derivatives_across_symmetry_axes(trK_dDD):
    print(" ", row)
print()

# Example 6d: Higher rank derivative object where only some components survive
par.set_parval_from_str("indexedexp::symmetry_axes", "0")

aDD_dDD = ixp.declarerank4("aDD_dDD", dimension=3, symmetry="sym01_sym23")
aDD_dDD_processed = ixp.zero_out_derivatives_across_symmetry_axes(aDD_dDD)

print("Symmetry axis = 0, on aDD_dDD; slice [0][1][:][:]:")
for row in aDD_dDD_processed[0][1]:
    print(" ", row)
print()

# Example 6e: Demonstrate error for unsupported higher order derivatives
par.set_parval_from_str("indexedexp::symmetry_axes", "2")
try:
    higher_deriv = ixp.declarerank3("trK_dDDD", dimension=3)
    ixp.zero_out_derivatives_across_symmetry_axes(higher_deriv)
except ValueError as e:
    print("As expected, order > 2 derivatives raise an error:")
    print(" ", e)

Rank 1 derivative object trK_dD with no symmetry axes:
  [trK_dD0, trK_dD1, trK_dD2]

Symmetry axis = 2, applied to trK_dD:
  [trK_dD0, trK_dD1, 0]

Symmetry axis = 2, applied to trK_dDD:
  [trK_dDD00, trK_dDD01, 0]
  [trK_dDD10, trK_dDD11, 0]
  [0, 0, 0]

Symmetry axes = 0,1, applied to trK_dDD:
  [0, 0, 0]
  [0, 0, 0]
  [0, 0, trK_dDD22]

Symmetry axis = 0, on aDD_dDD; slice [0][1][:][:]:
  [0, 0, 0]
  [0, aDD_dDD0111, aDD_dDD0112]
  [0, aDD_dDD0112, aDD_dDD0122]

As expected, order > 2 derivatives raise an error:
  Error. Derivative order > 2 not supported. Failed expression: trK_dDDD000


# Step 7: Inverting matrices with symmetric and generic matrix inverters
### \[Back to [top](#Table-of-Contents)\]

Matrix inversion is needed everywhere, for example when converting between
covariant and contravariant components of a metric.

SymPy can invert matrices, but `indexedexp.py` provides faster hand written
inversion routines, both for symmetric and generic matrices:

- Symmetric matrices:
  - `ixp.symm_matrix_inverter2x2(a)`
  - `ixp.symm_matrix_inverter3x3(a)`
  - `ixp.symm_matrix_inverter4x4(a)`
- Generic matrices:
  - `ixp.generic_matrix_inverter2x2(a)`
  - `ixp.generic_matrix_inverter3x3(a)`
  - `ixp.generic_matrix_inverter4x4(a)`

Each function returns `(inverse, determinant)` and raises
`ixp.NonInvertibleMatrixError` if the determinant is zero.

In [7]:
# Step 7: Matrix inversion helpers

def matmul(A, B):
    """
    Multiply two nested-list matrices A and B.
    """
    nrows = len(A)
    ncols = len(B[0])
    nmid = len(B)
    out = [[sp.sympify(0) for _ in range(ncols)] for __ in range(nrows)]
    for i in range(nrows):
        for j in range(ncols):
            s = sp.sympify(0)
            for k in range(nmid):
                s += A[i][k] * B[k][j]
            out[i][j] = sp.simplify(s)
    return out

def print_matrix(M, label):
    print(label)
    for row in M:
        print(" ", row)
    print()

# Example 7a: symmetric 2 by 2 numeric matrix
a = [[2, 1],
     [1, 3]]
inv_a, det_a = ixp.symm_matrix_inverter2x2(a)
print_matrix(a, "Original symmetric 2 by 2 matrix a:")
print("Determinant det(a) =", det_a)
print_matrix(inv_a, "Inverse matrix inv(a):")
I_test = matmul(inv_a, a)
print_matrix(I_test, "inv(a) * a (should be identity):")

# Example 7b: generic 2 by 2 symbolic matrix
b00, b01, b10, b11 = sp.symbols("b00 b01 b10 b11")
b = [[b00, b01],
     [b10, b11]]
ginv, gdet = ixp.generic_matrix_inverter2x2(b)
print("Generic 2 by 2 determinant:")
print(" ", gdet)
print_matrix(ginv, "Generic 2 by 2 inverse matrix:")

# Example 7c: symmetric 3 by 3 metric-like matrix
gDD_sym = ixp.declarerank2("gDD", dimension=3, symmetry="sym01")
gUU_sym, detg = ixp.symm_matrix_inverter3x3(gDD_sym)
print("Symmetric 3 by 3 metric gDD_det symbol:")
print(" ", detg)
print("Sample inverse components gamma^ij:")
print("  gUU_sym[0][0] =", gUU_sym[0][0])
print("  gUU_sym[0][1] =", gUU_sym[0][1])
print()

# Example 7d: singular matrix and NonInvertibleMatrixError
singular = [[1, 2],
            [2, 4]]  # determinant is zero
try:
    inv_sing, det_sing = ixp.symm_matrix_inverter2x2(singular)
except ixp.NonInvertibleMatrixError as e:
    print("Attempting to invert a singular matrix correctly raises:")
    print(" ", e)

Original symmetric 2 by 2 matrix a:
  [2, 1]
  [1, 3]

Determinant det(a) = 5
Inverse matrix inv(a):
  [0.6, -0.2]
  [-0.2, 0.4]

inv(a) * a (should be identity):
  [1.00000000000000, -1.11022302462516e-16]
  [0, 1.00000000000000]

Generic 2 by 2 determinant:
  b00*b11 - b01*b10
Generic 2 by 2 inverse matrix:
  [b11/(b00*b11 - b01*b10), -b01/(b00*b11 - b01*b10)]
  [-b10/(b00*b11 - b01*b10), b00/(b00*b11 - b01*b10)]

Symmetric 3 by 3 metric gDD_det symbol:
  gDD00*gDD11*gDD22 - gDD00*gDD12**2 - gDD01**2*gDD22 + 2*gDD01*gDD02*gDD12 - gDD02**2*gDD11
Sample inverse components gamma^ij:
  gUU_sym[0][0] = (gDD11*gDD22 - gDD12**2)/(gDD00*gDD11*gDD22 - gDD00*gDD12**2 - gDD01**2*gDD22 + 2*gDD01*gDD02*gDD12 - gDD02**2*gDD11)
  gUU_sym[0][1] = (-gDD01*gDD22 + gDD02*gDD12)/(gDD00*gDD11*gDD22 - gDD00*gDD12**2 - gDD01**2*gDD22 + 2*gDD01*gDD02*gDD12 - gDD02**2*gDD11)

Attempting to invert a singular matrix correctly raises:
  matrix has determinant zero


# Step 8: Levi-Civita symbols and tensors in 3 dimensions
### \[Back to [top](#Table-of-Contents)\]

The Levi-Civita symbol captures orientation and volume information in 3 dimensions.
`indexedexp.py` provides three related helpers:

1. `ixp.LeviCivitaSymbol_dim3_rank3()`:
   - Returns the combinatorial symbol $\epsilon_{ijk}$ with values in $\{-1, 0, 1\}$.
2. `ixp.LeviCivitaTensorUUU_dim3_rank3(sqrtgammaDET)`:
   - Returns the tensor with raised indices
     $\epsilon^{ijk} = \epsilon_{ijk} / \sqrt{\gamma}$.
3. `ixp.LeviCivitaTensorDDD_dim3_rank3(sqrtgammaDET)`:
   - Returns the tensor with lowered indices
     $\epsilon_{ijk} = \epsilon_{ijk} \sqrt{\gamma}$.

A very common use is to compute cross products:

$$(a \times b)^i = \sum_{j,k} \epsilon^{ijk} a_j b_k.$$

In [8]:
# Step 8: Levi-Civita symbols and tensors

# Example 8a: raw symbol
eps_symbol = ixp.LeviCivitaSymbol_dim3_rank3()
print("Levi-Civita symbol epsilon_ijk (selected entries):")
print("  epsilon_012 =", eps_symbol[0][1][2])
print("  epsilon_021 =", eps_symbol[0][2][1])
print("  epsilon_111 =", eps_symbol[1][1][1])
print()

# Example 8b: tensor with upper indices
sqrtgammaDET = sp.Symbol("sqrtgammaDET")
eps_UUU = ixp.LeviCivitaTensorUUU_dim3_rank3(sqrtgammaDET)
print("Levi-Civita tensor with raised indices:")
print("  epsilon^012 =", eps_UUU[0][1][2])
print("  epsilon^021 =", eps_UUU[0][2][1])
print()

# Example 8c: tensor with lower indices
eps_DDD = ixp.LeviCivitaTensorDDD_dim3_rank3(sqrtgammaDET)
print("Levi-Civita tensor with lowered indices:")
print("  epsilon_012 =", eps_DDD[0][1][2])
print("  epsilon_021 =", eps_DDD[0][2][1])
print()

# Example 8d: use the symbol to compute and verify a cross product

# Define abstract components for two vectors a and b
a_vec = [sp.Symbol("a0"), sp.Symbol("a1"), sp.Symbol("a2")]
b_vec = [sp.Symbol("b0"), sp.Symbol("b1"), sp.Symbol("b2")]

# Compute (a x b)^i using epsilon_ijk
cross_eps = [sp.sympify(0) for _ in range(3)]
for i in range(3):
    for j in range(3):
        for k in range(3):
            cross_eps[i] += eps_symbol[i][j][k] * a_vec[j] * b_vec[k]

print("Cross product using Levi-Civita symbol:")
for i in range(3):
    print(f"  (a x b)^{i} =", sp.simplify(cross_eps[i]))

# Verify against the standard formula
cx0 = a_vec[1]*b_vec[2] - a_vec[2]*b_vec[1]
cx1 = a_vec[2]*b_vec[0] - a_vec[0]*b_vec[2]
cx2 = a_vec[0]*b_vec[1] - a_vec[1]*b_vec[0]
standard_cross = [cx0, cx1, cx2]

print("\nVerification: difference between epsilon-based and standard formula:")
for i in range(3):
    diff = sp.simplify(cross_eps[i] - standard_cross[i])
    print(f"  component {i}: diff =", diff)

Levi-Civita symbol epsilon_ijk (selected entries):
  epsilon_012 = 1
  epsilon_021 = -1
  epsilon_111 = 0

Levi-Civita tensor with raised indices:
  epsilon^012 = 1/sqrtgammaDET
  epsilon^021 = -1/sqrtgammaDET

Levi-Civita tensor with lowered indices:
  epsilon_012 = sqrtgammaDET
  epsilon_021 = -sqrtgammaDET

Cross product using Levi-Civita symbol:
  (a x b)^0 = a1*b2 - a2*b1
  (a x b)^1 = -a0*b2 + a2*b0
  (a x b)^2 = a0*b1 - a1*b0

Verification: difference between epsilon-based and standard formula:
  component 0: diff = 0
  component 1: diff = 0
  component 2: diff = 0


# Step 9: Worked example: 3-metric, its inverse, and Levi-Civita tensor
### \[Back to [top](#Table-of-Contents)\]

In this final step we combine several tools from the notebook in a workflow that
looks very similar to what appears in numerical relativity:

1. Declare a symmetric spatial metric $\gamma_{ij}$.
2. Invert it to obtain $\gamma^{ij}$ and $\det(\gamma)$.
3. Construct a Levi-Civita tensor with raised indices using $\sqrt{\det(\gamma)}$.
4. Use it to form the cross product of two covariant vectors.

In [9]:
# Step 9: 3-metric, its inverse, and Levi-Civita tensor

# 1. Declare a symmetric 3 metric gamma_ij
gammaDD = ixp.declarerank2("gammaDD", dimension=3, symmetry="sym01")
print("3 metric gammaDD (symbolic):")
for row in gammaDD:
    print(" ", row)
print()

# 2. Invert gamma_ij to get gamma^ij and det(gamma)
gammaUU, detgamma = ixp.symm_matrix_inverter3x3(gammaDD)
print("Determinant det(gamma) =", detgamma)
print("Sample components of gammaUU:")
print("  gammaUU[0][0] =", gammaUU[0][0])
print("  gammaUU[0][1] =", gammaUU[0][1])
print()

# 3. Construct epsilon^ijk using sqrt(detgamma)
sqrtgamma = sp.sqrt(detgamma)
epsUUU = ixp.LeviCivitaTensorUUU_dim3_rank3(sqrtgamma)
print("Levi-Civita tensor epsilon^ijk built from det(gamma):")
print("  epsilon^012 =", epsUUU[0][1][2])
print("  epsilon^120 =", epsUUU[1][2][0])
print()

# 4. Define two covariant vectors A_i and B_i and compute (A x B)^i
A_D = ixp.declarerank1("A_D", dimension=3)
B_D = ixp.declarerank1("B_D", dimension=3)

# Raise indices using gamma^ij: A^i = gamma^ij A_j, and similarly for B^i
A_U = [sp.sympify(0) for _ in range(3)]
B_U = [sp.sympify(0) for _ in range(3)]
for i in range(3):
    for j in range(3):
        A_U[i] += gammaUU[i][j] * A_D[j]
        B_U[i] += gammaUU[i][j] * B_D[j]

# Compute (A x B)^i = epsilon^ijk A_j B_k (here we treat A_U, B_U as contravariant)
cross_U = [sp.sympify(0) for _ in range(3)]
for i in range(3):
    for j in range(3):
        for k in range(3):
            cross_U[i] += epsUUU[i][j][k] * A_U[j] * B_U[k]

print("Symbolic cross product (A x B)^i in terms of gamma^ij, A_D, and B_D:")
for i in range(3):
    print(f"  (A x B)^{i} =", sp.simplify(cross_U[i]))

3 metric gammaDD (symbolic):
  [gammaDD00, gammaDD01, gammaDD02]
  [gammaDD01, gammaDD11, gammaDD12]
  [gammaDD02, gammaDD12, gammaDD22]

Determinant det(gamma) = gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11
Sample components of gammaUU:
  gammaUU[0][0] = (gammaDD11*gammaDD22 - gammaDD12**2)/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)
  gammaUU[0][1] = (-gammaDD01*gammaDD22 + gammaDD02*gammaDD12)/(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)

Levi-Civita tensor epsilon^ijk built from det(gamma):
  epsilon^012 = 1/sqrt(gammaDD00*gammaDD11*gammaDD22 - gammaDD00*gammaDD12**2 - gammaDD01**2*gammaDD22 + 2*gammaDD01*gammaDD02*gammaDD12 - gammaDD02**2*gammaDD11)
  epsilon^120 = 1/sqrt(gammaDD00*gammaDD11*gammaDD